In [1]:
import os 
os.chdir('../../../')
print("Current working directory is now: ", os.getcwd())

import pandas as pd 
import numpy as np
import csv
import utils.interpretable_functions as interpret
import utils.RiskSLIM as slim
import utils.stumps as stumps
import utils.fairness_functions as fairness
from utils.load_settings import load_settings

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.metrics import roc_auc_score

from pprint import pprint
from riskslim.helper_functions import load_data_from_csv, print_model

# restore saved variables
%store -r summary_M6_FL_interpret

Current working directory is now:  C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2


C:\Users\binha\Anaconda3\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



### CART & EBM

In [2]:
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
x = data.loc[:,:'five_year']
y = data['misdemeanor_six_month'].values

In [3]:
#### CART
depth = [1,2,3,4,5]
impurity = [0.001, 0.002,0.003, 0.004, 0.005]
cart_summary = interpret.CART(X=x,
                         Y=y,
                         depth=depth,
                         impurity=impurity, 
                         seed = 816)

#### EBM
estimators = [40,60,80,100]
depth = [1,2,3]
learning_rate = [0.01]
holdout_split = [0.7, 0.9]
ebm_summary = interpret.EBM(X=x,
                       Y=y,
                       learning_rate = learning_rate,
                       depth = depth,
                       estimators=estimators,
                       holdout_split=holdout_split,
                       seed=816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binh

In [6]:
np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])

(0.5402375592113531, 0.06371592953065544)

In [7]:
np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])

(0.6115315243912367, 0.09909772154758092)

## Lasso Stumps

In [8]:
## load stumps & original data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv").sort_values('person_id')
original_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_data.csv").sort_values('person_id')
original_data = original_data.loc[:, ['person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
data = pd.merge(original_data, data, on=['person_id', 'screening_date'])

## split X & Y
X_stumps = data.loc[:,:'five_year1'].copy()
Y_stumps = data['misdemeanor_six_month'].values.copy()
Y_stumps[Y_stumps == -1] = 0
cols = X_stumps.columns[5:]

## load train & test stumps data
train_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_train_stumps.csv")
test_stumps = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_test_stumps.csv")
X_train_stumps = train_stumps.loc[:,:'five_year1'].copy()
X_test_stumps = test_stumps.loc[:,:'five_year1'].copy()
Y_train_stumps = train_stumps['misdemeanor_six_month'].values.copy()
Y_test_stumps = test_stumps['misdemeanor_six_month'].values.copy()
Y_train_stumps[Y_train_stumps == -1] = 0
Y_test_stumps[Y_test_stumps == -1] = 0

##### Single Stump Model

In [11]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.08, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

23

In [12]:
single_stump_model['features']

['age_at_current_charge19',
 'age_at_current_charge24',
 'age_at_current_charge37',
 'age_at_first_charge22',
 'age_at_first_charge30',
 'age_at_first_charge48',
 'p_charges3',
 'p_incarceration1',
 'p_weapon1',
 'p_fta_two_year1',
 'current_violence1',
 'p_felony2',
 'p_misdemeanor5',
 'p_misdemeanor6',
 'p_violence3',
 'p_arrest3',
 'p_property2',
 'p_property3',
 'p_stealing1',
 'p_stealing3',
 'six_month1',
 'one_year1',
 'three_year1']

#### Nested Cross Validation

In [13]:
stump_summary = stumps.stump_cv(X = X_stumps, 
                                Y = Y_stumps, 
                                columns=cols, 
                                c_grid={'C': [0.01, 0.02, 0.03, 0.04, 0.05, \
                                              0.06, 0.07, 0.08]}, 
                                seed = 816)

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [14]:
stump_summary['best_params'], np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])

([{'C': 0.07}, {'C': 0.06}, {'C': 0.06}, {'C': 0.08}, {'C': 0.02}],
 0.5811777474694112,
 0.06432816081073973)

##### Best Stump Model

In [15]:
best_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.06, 
                                      columns=cols, 
                                      seed=816)
best_stump_model['test_auc']

0.6129929977856245

## RiskSLIM -- Unconstrained

In [32]:
## load stumps data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_stumps.csv").sort_values('person_id')

## single stump model
single_stump_model = stumps.stump_model(X_train_stumps, 
                                        Y_train_stumps, 
                                        X_test_stumps, 
                                        Y_test_stumps, 
                                        c=0.05, 
                                        columns=cols, 
                                        seed=816)
len(single_stump_model['features'])

10

In [34]:
selected_features = single_stump_model['features']

In [35]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['misdemeanor_six_month', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['misdemeanor_six_month', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [36]:
riskslim_summary = slim.risk_nested_cv(X=sub_X, 
                                       Y=sub_Y, 
                                       indicator = indicator,
                                       y_label='misdemeanor_six_month', 
                                       max_coef=5, 
                                       max_coef_number=5, 
                                       max_runtime=1000,
                                       max_offset=100,
                                       c=1e-6, 
                                       seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/21/19 @ 10:51 PM | 153 rows in lookup table
11/21/19 @ 10:51 PM | ------------------------------------------------------------
11/21/19 @ 10:51 PM | runnning initialization procedure
11/21/19 @ 10:51 PM | ------------------------------------------------------------
11/21/19 @ 10:51 PM | CPA produced 2 cuts
11/21/19 @ 10:51 PM | running naive rounding on 4 solutions
11/21/19 @ 10:51 PM | best objective value: 0.6931
11/21/19 @ 10:51 PM | rounding produced 4 integer solutions
11/21/19 @ 10:51 PM | best objective value is 0.6931
11/21/19 @ 10:51 PM | running sequential rounding on 4 solutions
11/21/19 @ 10:51 PM | best objective value: 0.6931
11/21/19 @ 10:51 PM | sequential rounding produced 0 integer solutions
11/21/19 @ 10:51 PM | polishing 4 solutions
11/21/19 @ 10:51 PM | best objective value: 0.6931
11/21/19 @ 10:51 PM | polishing produced 4 integer solutions
11/21/19 @ 10:51 PM | initialization produced 7 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:51 PM | adding 223 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3765.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:51 PM | adding 207 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3893.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:51 PM | adding 205 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3749.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:51 PM | adding 208 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3795.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:53 PM | adding 201 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3747.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:53 PM | adding 201 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3635.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:53 PM | adding 193 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3630.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:53 PM | adding 197 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3650.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [37]:
np.mean(riskslim_summary['validation_auc']), np.mean(riskslim_summary['test_auc'])

(0.5855188651553379, 0.5866392569061963)

### Single RiskSLIM Model -- Unconstrained

In [38]:
selected_features = ["misdemeanor_six_month"] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [39]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'misdemeanor_six_month',
    'sample_weights': sample_weights
}

In [40]:
model_info, mip_info, lcpa_info = slim.risk_slim(new_train_data, 
                                                 max_coefficient=5, 
                                                 max_L0_value=5, 
                                                 c0_value=1e-6, 
                                                 max_offset=100, 
                                                 max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
11/21/19 @ 10:56 PM | 153 rows in lookup table
11/21/19 @ 10:56 PM | ------------------------------------------------------------
11/21/19 @ 10:56 PM | runnning initialization procedure
11/21/19 @ 10:56 PM | ------------------------------------------------------------
11/21/19 @ 10:56 PM | CPA produced 2 cuts
11/21/19 @ 10:56 PM | running naive rounding on 3 solutions
11/21/19 @ 10:56 PM | best objective value: 0.6931
11/21/19 @ 10:56 PM | rounding produced 3 integer solutions
11/21/19 @ 10:56 PM | best objective value is 0.6931
11/21/19 @ 10:56 PM | running sequential rounding on 3 solutions
11/21/19 @ 10:56 PM | best objective value: 0.6931
11/21/19 @ 10:56 PM | sequential rounding produced 0 integer solutions
11/21/19 @ 10:56 PM | polishing 3 solutions
11/21/19 @ 10:56 PM | best objective value: 0.6931
11/21/19 @ 10:56 PM | polishing produced 3 integer solutions
11/21/19 @ 10:56 PM | initialization produced 5 feasible soluti

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 10:56 PM | adding 206 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3776.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 24 columns, and 63 nonzeros.
Reduced MIP has 10 binaries, 12 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.03 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [41]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [42]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.6193907387516409

In [43]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.564276737087797

## RiskSLIM -- Constrained

In [143]:
single_stump_model = stumps.stump_model(X_train_stumps, 
                                      Y_train_stumps, 
                                      X_test_stumps, 
                                      Y_test_stumps, 
                                      c=0.07,     
                                      columns=cols, 
                                      seed=816)
len(single_stump_model['features'])

20

In [144]:
selected_features = single_stump_model['features']

In [145]:
### Subset features
if 'sex1' in selected_features:
    selected_features = ['misdemeanor_six_month', 'person_id', 'screening_date', 'race'] + selected_features
    indicator = 1
else:
    selected_features = ['misdemeanor_six_month', 'person_id', 'screening_date', 'race', 'sex1'] + selected_features
    indicator = 0

sub_data = data[selected_features]
sub_data = pd.merge(sub_data, original_data, on=['person_id', 'screening_date'])
sub_X, sub_Y = sub_data.iloc[:,1:], sub_data.iloc[:,0].values
sub_X.insert(0, '(Intercept)', 1)

In [146]:
riskslim_cs_summary = slim.risk_nested_cv_constrain(X=sub_X, 
                                                    Y=sub_Y, 
                                                    indicator = indicator,
                                                    y_label='misdemeanor_six_month', 
                                                    max_coef=5, 
                                                    max_coef_number=5, 
                                                    max_runtime=1000,
                                                    max_offset=100,
                                                    c=1e-6, 
                                                    seed=816)

setting c0 = 0.0 to ensure that intercept is not penalized
11/21/19 @ 11:18 PM | 253 rows in lookup table
11/21/19 @ 11:18 PM | ------------------------------------------------------------
11/21/19 @ 11:18 PM | runnning initialization procedure
11/21/19 @ 11:18 PM | ------------------------------------------------------------
11/21/19 @ 11:18 PM | CPA produced 2 cuts
11/21/19 @ 11:18 PM | running naive rounding on 27 solutions
11/21/19 @ 11:18 PM | best objective value: 0.3681
11/21/19 @ 11:18 PM | rounding produced 5 integer solutions
11/21/19 @ 11:18 PM | best objective value is 0.3730
11/21/19 @ 11:18 PM | running sequential rounding on 27 solutions
11/21/19 @ 11:18 PM | best objective value: 0.3681
11/21/19 @ 11:18 PM | sequential rounding produced 6 integer solutions
11/21/19 @ 11:18 PM | best objective value: 0.3730
11/21/19 @ 11:18 PM | polishing 11 solutions
11/21/19 @ 11:18 PM | best objective value: 0.3730
11/21/19 @ 11:18 PM | polishing produced 4 integer solutions
11/21/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:18 PM | adding 126 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3730.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:18 PM | adding 158 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3825.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:18 PM | adding 148 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3699.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:18 PM | adding 154 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3748.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

...

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:19 PM | adding 156 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3696.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:19 PM | adding 153 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3632.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:20 PM | adding 175 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3539.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:20 PM | adding 159 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3650.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

divide by zero encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:43: RuntimeWarning:

invalid value encountered in longlong_scalars

C:\Users\binha\Documents\Duke\Cynthia Research\psa-analysis - test2\utils\fairness_functions.py:88: RuntimeWarning:

invalid value encountered in longlong_scalars



In [147]:
np.mean(riskslim_cs_summary['validation_auc']), np.mean(riskslim_cs_summary['test_auc'])

(0.566869323321162, 0.5777788314130614)

### Single RiskSLIM Model -- Constrained

In [148]:
selected_features = ["misdemeanor_six_month"] + single_stump_model['features']
sub_train_data = train_stumps[selected_features]
sub_test_data = test_stumps[selected_features]

## split x 
sub_train_X = sub_train_data.iloc[:,1:]
sub_train_X.insert(0, '(Intercept)', 1)
sub_cols = sub_train_X.columns.tolist()
sub_train_X = sub_train_X.values
sub_test_X = sub_test_data.iloc[:,1:].values

## split y
sub_train_Y = sub_train_data.iloc[:,0].values.reshape(-1,1)
sub_test_Y = sub_test_data.iloc[:,0].values.reshape(-1,1)

## sample weight
sample_weights = np.repeat(1, len(sub_train_Y))

In [149]:
new_train_data = {
    'X': sub_train_X,
    'Y': sub_train_Y,
    'variable_names': sub_cols,
    'outcome_name': 'misdemeanor_six_month',
    'sample_weights': sample_weights
}

In [150]:
model_info, mip_info, lcpa_info = slim.risk_slim_constrain(new_train_data, 
                                                           max_coefficient=5, 
                                                           max_L0_value=5, 
                                                           c0_value=1e-6, 
                                                           max_offset=100, 
                                                           max_runtime=1000)
print_model(model_info['solution'], new_train_data)

setting c0 = 0.0 to ensure that intercept is not penalized
11/21/19 @ 11:20 PM | 253 rows in lookup table
11/21/19 @ 11:20 PM | ------------------------------------------------------------
11/21/19 @ 11:20 PM | runnning initialization procedure
11/21/19 @ 11:20 PM | ------------------------------------------------------------
11/21/19 @ 11:20 PM | CPA produced 2 cuts
11/21/19 @ 11:20 PM | running naive rounding on 29 solutions
11/21/19 @ 11:20 PM | best objective value: 0.3730
11/21/19 @ 11:20 PM | rounding produced 5 integer solutions
11/21/19 @ 11:20 PM | best objective value is 0.3758
11/21/19 @ 11:20 PM | running sequential rounding on 29 solutions
11/21/19 @ 11:20 PM | best objective value: 0.3730
11/21/19 @ 11:20 PM | sequential rounding produced 6 integer solutions
11/21/19 @ 11:20 PM | best objective value: 0.3730
11/21/19 @ 11:20 PM | polishing 11 solutions
11/21/19 @ 11:20 PM | best objective value: 0.3730
11/21/19 @ 11:20 PM | polishing produced 5 integer solutions
11/21/19 

Lazy constraint(s) or lazy constraint callback is present.
    Disabling dual reductions (CPX_PARAM_REDUCE) in presolve.
    Disabling non-linear reductions (CPX_PARAM_PRELINEAR) in presolve.
         Disabling repeat represolve because of lazy constraint/incumbent callback.
11/21/19 @ 11:20 PM | adding 183 initial cuts
1 of 1 MIP starts provided solutions.
MIP start 'mip_start_0' defined initial solution with objective 0.3730.
Tried aggregator 1 time.
Reduced MIP has 22 rows, 44 columns, and 83 nonzeros.
Reduced MIP has 20 binaries, 22 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.00 sec. (0.04 ticks)
Probing time = 0.00 sec. (0.01 ticks)
MIP emphasis: balance optimality and feasibility.
MIP search method: traditional branch-and-cut.
Parallel mode: none, using 1 thread.
Root relaxation solution time = 0.00 sec. (0.03 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap         Variabl

In [151]:
sub_train_X = sub_train_X[:,1:]
sub_train_Y[sub_train_Y == -1] = 0
sub_test_Y[sub_test_Y == -1] = 0

In [152]:
roc_auc_score(sub_train_Y, slim.riskslim_prediction(sub_train_X, np.array(cols), model_info).reshape(-1,1))

0.5961947278911566

In [153]:
roc_auc_score(sub_test_Y, slim.riskslim_prediction(sub_test_X, np.array(cols), model_info).reshape(-1,1))

0.5513196480938416

## Arnold PSA

In [154]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['arnold_nca', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['misdemeanor_six_month'].values

In [155]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
arnold_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    arnold_auc.append(roc_auc_score(test_y, test_x['arnold_nca'].values))
    
    ################################
    ## race_auc
    try:
        arnold_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['arnold_nca'],
                                                   labels = test_y)
        arnold_race_auc_final = arnold_race_auc.assign(fold_num = [i]*arnold_race_auc['Attribute'].count())
        race_auc.append(arnold_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    arnold_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['arnold_nca'],
                                                                         labels = test_y)
    arnold_condition_pn_final = arnold_condition_pn.assign(fold_num = [i]*arnold_condition_pn['Attribute'].count())
    condition_pn.append(arnold_condition_pn_final)
    
    ################################
    ## no condition pn
    arnold_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['arnold_nca'],
                                                                labels = test_y)
    arnold_no_condition_pn_final = arnold_no_condition_pn.assign(fold_num = [i]*arnold_no_condition_pn['Attribute'].count())
    no_condition_pn.append(arnold_no_condition_pn_final)
    
    i += 1
    
## race_auc
arnold_race_auc = []
try:
    arnold_race_auc = pd.concat(race_auc, ignore_index=True)
    arnold_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    arnold_race_auc = arnold_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
arnold_condition_pn = pd.concat(condition_pn, ignore_index=True)
arnold_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_condition_pn = arnold_condition_pn.reset_index(drop=True)

## no_condition_pn
arnold_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
arnold_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
arnold_no_condition_pn = arnold_no_condition_pn.reset_index(drop=True)

### Single Arnold PSA

In [156]:
test_data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold_test.csv")
X = test_data['arnold_nca'].values
Y = test_data['misdemeanor_six_month'].values
roc_auc_score(Y, X)

0.5549703752468729

## Compass

In [157]:
### load data
data = pd.read_csv("~/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/data/broward_arnold.csv").sort_values('person_id')
X_arnold = data.loc[:,['Risk of Recidivism_decile_score', 'sex', 'race', 'person_id', 'screening_date', 'age_at_current_charge', 'p_charges']]
Y_arnold = data['misdemeanor_six_month'].values

In [158]:
## set up cross validation
cv = KFold(n_splits=5,shuffle=True,random_state=816)
compas_auc = []
race_auc = []
condition_pn = []
no_condition_pn = []

i = 0
for train, test in cv.split(X_arnold, Y_arnold):
    
    train_x, train_y = X_arnold.iloc[train], Y_arnold[train]
    test_x, test_y = X_arnold.iloc[test], Y_arnold[test]
    holdout_with_attrs = test_x.copy()
    
    ################################
    ## arnold_auc
    compas_auc.append(roc_auc_score(test_y, test_x['Risk of Recidivism_decile_score'].values))
    
    ################################
    ## race_auc
    try:
        compas_race_auc = fairness.fairness_in_auc(df = holdout_with_attrs,
                                                   probs = test_x['Risk of Recidivism_decile_score'],
                                                   labels = test_y)
        compas_race_auc_final = compas_race_auc.assign(fold_num = [i]*compas_race_auc['Attribute'].count())
        race_auc.append(compas_race_auc_final)
    except:
        pass
    
    ################################
    ## condition pn
    compas_condition_pn = fairness.conditional_balance_positive_negative(df = holdout_with_attrs,
                                                                         probs = test_x['Risk of Recidivism_decile_score'],
                                                                         labels = test_y)
    compas_condition_pn_final = compas_condition_pn.assign(fold_num = [i]*compas_condition_pn['Attribute'].count())
    condition_pn.append(compas_condition_pn_final)
    
    ################################
    ## no condition pn
    compas_no_condition_pn = fairness.balance_positive_negative(df = holdout_with_attrs,
                                                                probs = test_x['Risk of Recidivism_decile_score'],
                                                                labels = test_y)
    compas_no_condition_pn_final = compas_no_condition_pn.assign(fold_num = [i]*compas_no_condition_pn['Attribute'].count())
    no_condition_pn.append(compas_no_condition_pn_final)
    
    i += 1
    
## race_auc
compas_race_auc = []
try:  
    compas_race_auc = pd.concat(race_auc, ignore_index=True)
    compas_race_auc.sort_values(["fold_num", "Attribute"], inplace=True)
    compas_race_auc = compas_race_auc.reset_index(drop=True)
except:
    pass

## condition_pn
compas_condition_pn = pd.concat(condition_pn, ignore_index=True)
compas_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
compas_condition_pn = compas_condition_pn.reset_index(drop=True)

## no_condition_pn
compas_no_condition_pn = pd.concat(no_condition_pn, ignore_index=True)
compas_no_condition_pn.sort_values(["fold_num", "Attribute"], inplace=True)
compas_no_condition_pn = compas_no_condition_pn.reset_index(drop=True)

### Results

In [159]:
#### save results
summary_M6_FL_interpret = {"CART": cart_summary,
                           "EBM": ebm_summary, 
                           'Lasso Stumps': stump_summary, 
                           'RiskSLIM': riskslim_summary, 
                           'RiskSLIM-Constrained': riskslim_cs_summary, 
                           'Arnold PSA': arnold_auc, 
                           'Compas': compas_auc}
%store summary_M6_FL_interpret

Stored 'summary_M6_FL_interpret' (dict)


In [168]:
results = [["CART", np.mean(cart_summary['holdout_test_auc']), np.mean(cart_summary['auc_diffs'])],
           ["EBM", np.mean(ebm_summary['holdout_test_auc']), np.mean(ebm_summary['auc_diffs'])], 
           ["Lasso Stumps", np.mean(stump_summary['holdout_test_auc']), np.mean(stump_summary['auc_diffs'])],
           ['RiskSLIM', np.mean(riskslim_summary['test_auc'])],
           ['RiskSLIM-Constrained', np.mean(riskslim_cs_summary['test_auc'])],
           ['Arnold PSA', round(np.mean(arnold_auc), 3)], 
           ['Compas', round(np.mean(compas_auc), 3)]]
results

[['CART', 0.5402375592113531, 0.06371592953065544],
 ['EBM', 0.6115315243912367, 0.09909772154758092],
 ['Lasso Stumps', 0.5811777474694112, 0.06432816081073973],
 ['RiskSLIM', 0.5866392569061963],
 ['RiskSLIM-Constrained', 0.5777788314130614],
 ['Arnold PSA', 0.556],
 ['Compas', 0.569]]

In [169]:
auc = [np.mean(cart_summary['holdout_test_auc']), 
       np.mean(ebm_summary['holdout_test_auc']), 
       np.mean(stump_summary['holdout_test_auc']), 
       np.mean(riskslim_summary['test_auc']),
       np.mean(riskslim_cs_summary['test_auc']),]

In [170]:
path = "C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/model results/Interpretable Models/Six Month/"
results = [["Misdemeanor", np.str((round(np.mean(cart_summary['holdout_test_auc']), 3))) + " (" + np.str(round(np.std(cart_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(ebm_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(ebm_summary['holdout_test_auc']), 3)) + ")", 
            np.str(round(np.mean(stump_summary['holdout_test_auc']),3)) + " (" + np.str(round(np.std(stump_summary['holdout_test_auc']), 3)) + ")",             
            np.str(round(np.mean(riskslim_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_summary['test_auc']), 3)) + ")", 
            np.str(round(np.mean(riskslim_cs_summary['test_auc']),3)) + " (" + np.str(round(np.std(riskslim_cs_summary['test_auc']), 3)) + ")", 
            round(np.max(auc) - np.min(auc), 3),
            np.str(round(np.mean(arnold_auc), 3)) + " (" + np.str(round(np.std(arnold_auc),3)) + ")", 
            np.str(round(np.mean(compas_auc), 3)) + " (" + np.str(round(np.std(compas_auc),3)) + ")"]]
with open(path + 'Interpretable Models Summary.csv', 'a') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows(results)

### Confusion Matrix

In [163]:
cart_confusion = cart_summary['confusion_matrix_stats']
ebm_confusion = ebm_summary['confusion_matrix_stats']
riskslim_confusion = riskslim_summary['confusion_matrix_stats']
riskslim_cs_confusion = riskslim_cs_summary['confusion_matrix_stats']
stumps_confusion = stump_summary['confusion_matrix_stats']

cart_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/misdemeanor/cart_confusion.csv', index=None,header=True)
ebm_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/misdemeanor/ebm_confusion.csv', index=None,header=True)
riskslim_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/misdemeanor/riskslim_confusion.csv', index=None,header=True)
riskslim_cs_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/misdemeanor/riskslim_cs_confusion.csv', index=None,header=True)
stumps_confusion.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/confusion/six-month/misdemeanor/stumps_confusion.csv', index=None,header=True)

### Calibration

In [164]:
cart_calibration = cart_summary['calibration_stats']
ebm_calibration = ebm_summary['calibration_stats']
riskslim_calibration = riskslim_summary['calibration_stats']
riskslim_cs_calibration = riskslim_cs_summary['calibration_stats']
stumps_calibration = stump_summary['calibration_stats']

## save results
cart_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/misdemeanor/cart_calibration.csv', index=None,header=True)
ebm_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/misdemeanor/ebm_calibration.csv', index=None,header=True)
riskslim_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/misdemeanor/riskslim_calibration.csv', index=None,header=True)
riskslim_cs_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/misdemeanor/riskslim_cs_calibration.csv', index=None,header=True)
stumps_calibration.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/calibration/six-month/misdemeanor/stumps_calibration.csv', index=None,header=True)

### Race AUC

In [165]:
cart_race_auc = cart_summary['race_auc']
ebm_race_auc = ebm_summary['race_auc']
riskslim_race_auc = riskslim_summary['race_auc']
riskslim_cs_race_auc = riskslim_cs_summary['race_auc']
stumps_race_auc = stump_summary['race_auc']

## save results
cart_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/misdemeanor/cart_race_auc.csv', index=None,header=True)
ebm_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/misdemeanor/ebm_race_auc.csv', index=None,header=True)
riskslim_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/misdemeanor/riskslim_race_auc.csv', index=None,header=True)
riskslim_cs_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/misdemeanor/riskslim_cs_race_auc.csv', index=None,header=True)
stumps_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/misdemeanor/stumps_race_auc.csv', index=None,header=True)
arnold_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/misdemeanor/arnold_race_auc.csv', index=None,header=True)
compas_race_auc.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/race-auc/six-month/misdemeanor/compas_race_auc.csv', index=None,header=True)

### Condition PN

In [166]:
cart_condition_pn = cart_summary['condition_pn']
ebm_condition_pn = ebm_summary['condition_pn']
riskslim_condition_pn = riskslim_summary['condition_pn']
riskslim_cs_condition_pn = riskslim_cs_summary['condition_pn']
stumps_condition_pn = stump_summary['condition_pn']

## save results
cart_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/misdemeanor/cart_condition_pn.csv', index=None,header=True)
ebm_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/misdemeanor/ebm_condition_pn.csv', index=None,header=True)
riskslim_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/misdemeanor/riskslim_condition_pn.csv', index=None,header=True)
riskslim_cs_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/misdemeanor/riskslim_cs_condition_pn.csv', index=None,header=True)
stumps_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/misdemeanor/stumps_condition_pn.csv', index=None,header=True)
arnold_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/misdemeanor/arnold_condition_pn.csv', index=None,header=True)
compas_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/condition-pn/six-month/misdemeanor/compas_condition_pn.csv', index=None,header=True)

### No Condition PN

In [167]:
cart_no_condition_pn = cart_summary['no_condition_pn']
ebm_no_condition_pn = ebm_summary['no_condition_pn']
riskslim_no_condition_pn = riskslim_summary['no_condition_pn']
riskslim_cs_no_condition_pn = riskslim_cs_summary['no_condition_pn']
stumps_no_condition_pn = stump_summary['no_condition_pn']

## save results
cart_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/misdemeanor/cart_no_condition_pn.csv', index=None,header=True)
ebm_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/misdemeanor/ebm_no_condition_pn.csv', index=None,header=True)
riskslim_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/misdemeanor/riskslim_no_condition_pn.csv', index=None,header=True)
riskslim_cs_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/misdemeanor/riskslim_cs_no_condition_pn.csv', index=None,header=True)
stumps_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/misdemeanor/stumps_no_condition_pn.csv', index=None,header=True)
arnold_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/misdemeanor/arnold_no_condition_pn.csv', index=None,header=True)
compas_no_condition_pn.to_csv(r'C:/Users/binha/Documents/Duke/Cynthia Research/KY-analysis-mytrials/broward/broward models/no-condition-pn/six-month/misdemeanor/compas_no_condition_pn.csv', index=None,header=True)